In [ ]:
## Importing required libraries
import sqlite3
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline

In [2]:
## Fetching data
#Connecting to database
database = 'database.sqlite'
conn = sqlite3.connect(database)

#Defining the number of jobs to be run in parallel during grid search
n_jobs = 1 #Insert number of parallel jobs here

#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
#player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
#team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)

#Reduce match data to fulfill run time requirements
rows = ["country_id", "league_id", "season", "stage", "date", "match_api_id", "home_team_api_id", 
        "away_team_api_id", "home_team_goal", "away_team_goal", "home_player_1", "home_player_2",
        "home_player_3", "home_player_4", "home_player_5", "home_player_6", "home_player_7", 
        "home_player_8", "home_player_9", "home_player_10", "home_player_11", "away_player_1",
        "away_player_2", "away_player_3", "away_player_4", "away_player_5", "away_player_6",
        "away_player_7", "away_player_8", "away_player_9", "away_player_10", "away_player_11"]

#Dictionary to convert player IDs to names
playerDict = {}
i = 0
for playerID in player_data['player_api_id']:
    playerDict[playerID] =  player_data['player_name'][i]
    i += 1

In [3]:
cols_to_remove = ["home_player_1", "home_player_2", "home_player_3", "home_player_4", 
                        "home_player_5", "home_player_6", "home_player_7", "home_player_8", 
                        "home_player_9", "home_player_10", "home_player_11", "away_player_1", 
                        "away_player_2", "away_player_3", "away_player_4", "away_player_5", 
                        "away_player_6", "away_player_7", "away_player_8", "away_player_9", 
                        "away_player_10", "away_player_11", "home_player_X1", "home_player_X2", 
                        "home_player_X3", "home_player_X4", "home_player_X5", "home_player_X6", 
                        "home_player_X7", "home_player_X8", "home_player_X9", "home_player_X10", 
                        "home_player_X11", "away_player_X1", "away_player_X2", "away_player_X3", 
                        "away_player_X4", "away_player_X5", "away_player_X6", "away_player_X7", 
                        "away_player_X8", "away_player_X9", "away_player_X10", "away_player_X11",
                        "home_player_Y1", "home_player_Y2", "home_player_Y3", "home_player_Y4", 
                        "home_player_Y5", "home_player_Y6", "home_player_Y7", "home_player_Y8", 
                        "home_player_Y9", "home_player_Y10", "home_player_Y11", "away_player_Y1", 
                        "away_player_Y2", "away_player_Y3", "away_player_Y4", "away_player_Y5", 
                        "away_player_Y6", "away_player_Y7", "away_player_Y8", "away_player_Y9", 
                        "away_player_Y10", "away_player_Y11"]

match_data.drop(cols_to_remove, axis=1, inplace=True)

In [ ]:
rows = ["home_team_goal", "away_team_goal", 'B365H', 'B365D',
       'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA']

print(match_data.shape)

#Figure out which bookies to remove
k = (match_data.dropna(subset=rows, inplace=False, how='any'))
print(k.shape)

In [ ]:
match_data.dropna(subset=rows, inplace=True, how='any')
def resultFinder(homeScore, awayScore):
    if (homeScore > awayScore):
        return 'Home Win'
    elif (homeScore == awayScore):
        return 'Draw'
    else: 
        return 'Home Loss'
match_data['result'] = pd.Series()
for i in range(len(match_data['home_team_goal'])):
    match_data['result'][i] = resultFinder(match_data['home_team_goal'][i], match_data['away_team_goal'][i])

In [ ]:
for i in range(len(match_data['home_team_goal'])):
    print('home:', match_data['home_team_goal'][i], '||away:', match_data['away_team_goal'][i], '||result:', match_data['result'][i])